# Feature extraction

In [31]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [2]:
data = pd.read_csv('data/data_split.csv', encoding='utf-8')

In [4]:
data.head()

,id,fandom1,fandom2,pair1,pair2
0,6cced668-6e51-5212-873c-717f2bc91ce6,Guardians of Ga'Hoole,Hetalia - Axis Powers,"I shift a bit, warily letting my eyes dart fro...","""All will become one with Russia,"" he said, al..."
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,Guardians of Ga'Hoole,Warriors,"I shift a bit, warily letting my eyes dart fro...","Suddenly, a piece of ice falls into the pit of..."
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,Guardians of Ga'Hoole,Xiaolin Showdown,A single tear escaped me as I left. I did have...,"got the Yang yoyo."" Kimiko pulled the other ha..."
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,Hetalia - Axis Powers,Warriors,"""Ja."" Ludwig kept his gaze upon her, solidly. ...",SilverGray lll...YellowRagged llll...GrayMilli...
4,4fe541af-912e-5a86-81a5-94c6d3891509,Hetalia - Axis Powers,Xiaolin Showdown,"And he did. Slowly, hesitantly...but coming fr...","""Let""s go,"" Raimondo said and then started in ..."


In [38]:
data

,id,fandom1,fandom2,pair1,pair2
0,6cced668-6e51-5212-873c-717f2bc91ce6,Guardians of Ga'Hoole,Hetalia - Axis Powers,"I shift a bit, warily letting my eyes dart fro...","""All will become one with Russia,"" he said, al..."
1,3c6c188a-db28-59aa-8c09-3d0f799ff579,Guardians of Ga'Hoole,Warriors,"I shift a bit, warily letting my eyes dart fro...","Suddenly, a piece of ice falls into the pit of..."
2,b0cfa94f-c9ec-5aa5-8331-a5a249b664cf,Guardians of Ga'Hoole,Xiaolin Showdown,A single tear escaped me as I left. I did have...,"got the Yang yoyo."" Kimiko pulled the other ha..."
3,e6e86e73-9a7b-58f2-a652-a17b4a1bcabf,Hetalia - Axis Powers,Warriors,"""Ja."" Ludwig kept his gaze upon her, solidly. ...",SilverGray lll...YellowRagged llll...GrayMilli...
4,4fe541af-912e-5a86-81a5-94c6d3891509,Hetalia - Axis Powers,Xiaolin Showdown,"And he did. Slowly, hesitantly...but coming fr...","""Let""s go,"" Raimondo said and then started in ..."
...,...,...,...,...,...
52596,3926a1bd-6d33-5513-a694-12d7590443d4,True Blood,Austin & Ally,Enjoy! And time stands still beneath the air o...,"""That a challenge, dick?"" Austin smirks back a..."
52597,17227d87-a3cb-5aca-b5fd-bb642e88b030,True Blood,Kuroko no Basuke/黒子のバスケ,"""I forgive you,"" she blurted out looking strai...","This is a KNB fanfic. KNB doesn""t belong to me..."
52598,0332dcaa-7230-508b-badb-820f9d8eebaa,Austin & Ally,Five Nights at Freddy´s,"""What? Why? What""s happening? Where are we goi...","At the moment, Chris"" position on the swing se..."
52599,f5e4884a-13fd-51fb-a880-053d6655d59a,Five Nights at Freddy´s,Austin & Ally,"""Gah!"" I growled. ""I need a password to access...",As you can see a lot of things has happened si...


## tri-punctuations

In [24]:

def punclist(text):
    punc_list = ''
    for i in text:
        if i == ',' or i =='.':
            punc_list+=i+' '
    return punc_list

In [19]:

punc12=pd.DataFrame(columns = ('one','two'))

In [22]:
punc12.loc[0]=[punc_list, punc_list]

In [33]:
%%time
punc12=pd.DataFrame(columns = ('one','two'))
for i in tqdm(range(len(data))):
    punc12.loc[i]=[punclist(data.pair1[i]), punclist(data.pair2[i])]

100%|██████████| 52601/52601 [07:07<00:00, 122.94it/s]

CPU times: user 6min 52s, sys: 8.1 s, total: 7min
Wall time: 7min 7s


In [34]:
punc12.to_csv('feature/punctuation_list.csv',index=False)

In [35]:
def ngram_punc(text, num):
    tmp = [] 
    words = text.split()
    for i in range(len(words)-num+1):
        tmp.append(words[i:i+num]) 
    return tmp

def diff_ngram_punc(text1, text2, num):
    
    a = ngram_punc(text1, num)
    b = ngram_punc(text2, num) 
    
    cnt = 0 
    for i in a:
        for j in b:
            if i == j:
                cnt += 1
                #common.append(i)
    if len(a) == 0:
        return 100000
    else:
        return cnt/len(a)


In [37]:
%%time
i = 1
diff_ngram_punc(punc12.one[i], punc12.two[i], 6)

CPU times: user 55.1 ms, sys: 1.87 ms, total: 57 ms
Wall time: 56.2 ms


57.66413043478261

In [79]:
i = 10
diff_ngram_punc(punc12.one[i], punc12.two[i], 3)

39.55347593582888

In [84]:
%%time
feature_punc3gram = np.zeros(len(data))
num = 3
for i in range(1):
    feature_punc3gram[i] = diff_ngram_punc(punc12.one[i], punc12.one[i], num)

CPU times: user 55.2 ms, sys: 1.73 ms, total: 56.9 ms
Wall time: 56.2 ms


In [104]:
%%time
feature_punc3gram = np.zeros(len(data))
num = 3
for i in tqdm(range(len(data))):
    feature_punc3gram[i] = diff_ngram_punc(data.pair1[i], data.pair2[i], num)

100%|██████████| 52601/52601 [8:55:26<00:00,  1.64it/s]    

CPU times: user 8h 48min 13s, sys: 1min 30s, total: 8h 49min 44s
Wall time: 8h 55min 26s


In [105]:
np.savetxt('feature/feature_punc3gram.txt',feature_punc3gram)

## average sentence length

In [37]:
len(data)

52601

In [3]:
def avg_sentence_length(text):
    sentences = re.split('[?|!|\.]',text)
    avg_sentence = np.mean(np.array([len(sentence) for sentence in sentences]))
    return avg_sentence

In [40]:
%%time
avg_sentence_len = np.zeros(len(data))
for i in range(len(data)):
    avg_sentence_len[i] = avg_sentence_length(data.pair1[i])/avg_sentence_length(data.pair2[i])

CPU times: user 26.9 s, sys: 76 ms, total: 27 s
Wall time: 27.1 s


In [43]:
np.savetxt('feature/feature_avg_sentence_length.txt',avg_sentence_len)
# avg_sentence_len = np.loadtxt('feature/avg_sentence_length.txt')

In [59]:
%%time
# 优化除法部分，精确分子分母大小
avg_sentence_len = np.zeros(len(data))
for i in range(len(data)):
    avgs1 = avg_sentence_length(data.pair1[i])
    avgs2 = avg_sentence_length(data.pair2[i])
    avg_sentence_len[i] = max(avgs1,avgs2)/min(avgs1,avgs2)

CPU times: user 26.9 s, sys: 85.6 ms, total: 27 s
Wall time: 27 s


In [60]:
np.savetxt('feature/feature_avg_sentence_length2max.txt',avg_sentence_len)


In [4]:
%%time
#feature_lexical_richness = np.zeros(len(data))
avg_sentence12 = np.zeros((len(data),2))
for i in range(len(data)):
    avgs1 = avg_sentence_length(data.pair1[i])
    avgs2 = avg_sentence_length(data.pair2[i])
    avg_sentence12[i][0] = avgs1
    avg_sentence12[i][1] = avgs2

CPU times: user 26.5 s, sys: 42.2 ms, total: 26.5 s
Wall time: 26.5 s


In [5]:
np.savetxt('feature/avg_sentence12.txt',avg_sentence12)

## average word length

In [62]:
def avg_word_length(text):
    text_clean = re.sub(r"[^\w]+", ' ', text)
    avg_word = np.mean(np.array([len(word) for word in nltk.word_tokenize(text_clean)]))
    return avg_word

In [59]:
%%time
avg_word_len = np.zeros(len(data))
for i in range(len(data)):
    avg_word_len[i] = avg_word_length(data.pair1[i])/avg_word_length(data.pair2[i])

CPU times: user 21min 32s, sys: 1.46 s, total: 21min 33s
Wall time: 21min 34s


In [61]:
np.savetxt('feature/feature_avg_word_length.txt',avg_word_len)
# avg_sentence_len = np.loadtxt('feature/avg_sentence_length.txt')

In [63]:
%%time
# 优化除法部分，精确分子分母大小
avg_word_len = np.zeros(len(data))
for i in range(len(data)):
    avgs1 = avg_word_length(data.pair1[i])
    avgs2 = avg_word_length(data.pair2[i])
    avg_word_len[i] = max(avgs1,avgs2)/min(avgs1,avgs2)

CPU times: user 21min 40s, sys: 1.77 s, total: 21min 42s
Wall time: 21min 44s


In [64]:
np.savetxt('feature/feature_avg_word_length2max.txt',avg_word_len)

## pos

In [ ]:
from collections import Counter

In [ ]:
POS_TAGS = [
                'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ',
                'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 
                'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$',
                'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH',
                'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT',
                'WP', 'WP$', 'WRB'
            ]

In [88]:

pos_list = ['ADJ','ADP','ADV','CONJ','DET','NOUN','NUM','PRT','PRON','VERB','.']
def pos_num(text):
    
    noun = 0
    pronoun = 0
    adjective = 0
    numeral = 0
    verb = 0
    adverb = 0
    end = 0
    comma = 0
    question = 0
    exclamation = 0
    total = len(text.split())
    tags = nltk.pos_tag(nltk.word_tokenize(text))
    counts = Counter( tag for word,  tag in tags)
    for tag in counts:
        if tag == 'NN' or tag == 'NNS' or tag == 'NNP' or tag == 'NNPS':
            noun+=counts[tag]
        elif tag == 'PRP':
            pronoun += counts[tag]
        elif tag == 'JJ' or tag == 'JJR 'or tag == 'JJS':
            adjective += counts[tag]
        elif tag == 'CD':
            numeral += counts[tag]
        elif tag == 'VB' or tag == 'VBD' or tag == 'VBG' or tag == 'VBN' or tag == 'VBP' or tag == 'VBZ':
            verb += counts[tag]
        elif tag == 'RB' or tag == 'RBR' or tag == 'RBS':
            adverb += counts[tag]
        elif tag == '.':
            end += counts[tag]
        elif tag == ',':
            comma += counts[tag]
        elif tag == '?':
            question += counts[tag]
        elif tag == '!':
            exclamation += counts[tag]
    #return noun,pronoun,adjective,numeral,verb,adverb,end,comma,question,exclamation
    return noun/total,pronoun/total,adjective/total,numeral/total,verb/total,adverb/total,end/total,comma/total
        

In [100]:
nounnum = avg_word_len = np.zeros(len(data))
pronounnum = avg_word_len = np.zeros(len(data))
adjectivenum = avg_word_len = np.zeros(len(data))
numeralnum = avg_word_len = np.zeros(len(data))
verbnum = avg_word_len = np.zeros(len(data)) 
adverbnum = avg_word_len = np.zeros(len(data))
endnum = avg_word_len = np.zeros(len(data))
commanum = avg_word_len = np.zeros(len(data))
questionnum = avg_word_len = np.zeros(len(data))
exclamationnum = avg_word_len = np.zeros(len(data))

In [101]:
%%time
for i in range(len(data)):
    noun1,pronoun1,adjective1,numeral1,verb1,adverb1,end1,comma1,question1,exclamation1 = pos_num(data.pair1[i])
    noun2,pronoun2,adjective2,numeral2,verb2,adverb2,end2,comma2,question2,exclamation2 = pos_num(data.pair2[i])
    nounnum[i] = noun1-noun2
    pronounnum[i] = pronoun1-pronoun2
    adjectivenum[i] = adjective1-adjective2
    numeralnum[i] = numeral1-numeral2
    verbnum[i] = verb1 - verb2
    adverbnum[i] = adverb1 - adverb2
    endnum[i] = end1 - end2
    commanum[i] = comma1 - comma2
    questionnum[i] = question1 - question2
    exclamationnum[i] = exclamation1 - exclamation2

CPU times: user 5h 22min 25s, sys: 23.7 s, total: 5h 22min 48s
Wall time: 5h 23min 8s


In [ ]:
np.savetxt('feature/nounnumddd.txt',nounnum)
np.savetxt('feature/pronounnumddd.txt',pronounnum)
np.savetxt('feature/adjectivenumddd.txt',adjectivenum)
np.savetxt('feature/numeralnumddd.txt',numeralnum)
np.savetxt('feature/verbnumddd.txt',verbnum)
np.savetxt('feature/adverbnumddd.txt',adverbnum)
np.savetxt('feature/endnumddd.txt',endnum)
np.savetxt('feature/commanumddd.txt',commanum)
np.savetxt('feature/questionnumddd.txt',questionnum) #0
np.savetxt('feature/exclamationnumddd.txt',exclamationnum) #0

# avg_sentence_len = np.loadtxt('feature/avg_sentence_length.txt')

In [103]:
np.savetxt('feature/nounnum.txt',nounnum)
np.savetxt('feature/pronounnum.txt',pronounnum)
np.savetxt('feature/adjectivenum.txt',adjectivenum)
np.savetxt('feature/numeralnum.txt',numeralnum)
np.savetxt('feature/verbnum.txt',verbnum)
np.savetxt('feature/adverbnum.txt',adverbnum)
np.savetxt('feature/endnum.txt',endnum)
np.savetxt('feature/commanum.txt',commanum)
np.savetxt('feature/questionnum.txt',questionnum) #0
np.savetxt('feature/exclamationnum.txt',exclamationnum) #0

# avg_sentence_len = np.loadtxt('feature/avg_sentence_length.txt')

In [110]:
nounnum

array([ 171.,  -90., -111., ..., -133.,  140.,  147.])

In [112]:
np.savetxt('feature/feature_noun.txt',abs(nounnum))
np.savetxt('feature/feature_pronoun.txt',abs(pronounnum))
np.savetxt('feature/feature_adjective.txt',abs(adjectivenum))
np.savetxt('feature/feature_numeral.txt',abs(numeralnum))
np.savetxt('feature/feature_verb.txt',abs(verbnum))
np.savetxt('feature/feature_adverb.txt',abs(adverbnum))
np.savetxt('feature/feature_end.txt',abs(endnum))
np.savetxt('feature/feature_comma.txt',abs(commanum))
np.savetxt('feature/feature_question.txt',abs(questionnum)) #0
np.savetxt('feature/feature_exclamation.txt',abs(exclamationnum)) #0

In [94]:
#test
%%time
i = 0
noun1,pronoun1,adjective1,numeral1,verb1,adverb1,end1,comma1,question1,exclamation1 = pos_num(data.pair1[i])
noun2,pronoun2,adjective2,numeral2,verb2,adverb2,end2,comma2,question2,exclamation2 = pos_num(data.pair2[i])
nounnum[i] = noun1-noun2
pronounnum[i] = pronoun1-pronoun2
adjectivenum[i] = adjective1-adjective2
numeralnum[i] = numeral1-numeral2
verbnum[i] = verb1 - verb2
adverbnum[i] = adverb1 - adverb2
endnum[i] = end1 - end2
commanum[i] = comma1 - comma2
questionnum[i] = question1 - question2
exclamationnum[i] = exclamation1 - exclamation2

CPU times: user 392 ms, sys: 8.86 ms, total: 400 ms
Wall time: 401 ms


In [98]:
#test
%%time
noun,pronoun,adjective,numeral,verb,adverb,end,comma,question,exclamation = pos_num(data.pair2[0])
(noun,pronoun,adjective,numeral,verb,adverb,end,comma,question,exclamation)

CPU times: user 197 ms, sys: 3.27 ms, total: 201 ms
Wall time: 200 ms


(818, 418, 268, 10, 813, 312, 237, 350, 0, 0)

# word number

In [124]:
np.array([[[['array'],['dd']],[['ef'],['ersr']]]])[0][0]

array([['array'],
       ['dd']], dtype='<U5')

## mispelling

In [117]:
from spellchecker import SpellChecker

In [148]:
def mispelling(text):
    spell = SpellChecker()
    text_clean = re.sub(r"[^\w]+", ' ', text)
    return len(spell.unknown(text_clean.split()))

In [160]:
%%time
total_mis = np.zeros((len(data),2))
feature_mis = np.zeros(len(data))
for i in range(len(data)):
    mis1 = mispelling(data.pair1[i])
    mis2 = mispelling(data.pair2[i])
    total_mis[i][0] = mis1
    total_mis[i][1] = mis2
    feature_mis[i] = abs(mis1 - mis2)

CPU times: user 3h 17min 14s, sys: 12min 50s, total: 3h 30min 4s
Wall time: 3h 30min 20s


In [164]:
feature_mis.shape

(52601,)

In [165]:
np.savetxt('feature/total_mispelling.txt',total_mis)
np.savetxt('feature/feature_mispelling.txt',feature_mis)


In [159]:
#test
%%time
i = 0
total_mis = np.zeros((len(data),2))
feature_mis = np.zeros(len(data))
mis1 = mispelling(data.pair1[i])
mis2 = mispelling(data.pair2[i])
total_mis[i][0] = mis1
total_mis[i][0] = mis2
feature_mis[i] = abs(mis1 - mis2)

CPU times: user 252 ms, sys: 20.1 ms, total: 272 ms
Wall time: 272 ms


## n-gram

In [7]:
def ngram_word(text, num):
    tmp = [] 
    words = text.split()
    for i in range(len(words)-num+1):
        tmp.append(words[i:i+num]) 
    return tmp

In [8]:
def ngram_letter(text, num):
    tmp = [] 
    words = text.split()
    for i in range(len(words)-num+1):
        tmp.append(words[i:i+num]) 
    return tmp

In [9]:
def ngram_word_mul(text, num1,num2,num3):
    tmp1 = []
    tmp2 = []
    tmp3 = []
    words = text.split()
    for i in range(len(words)-num3+1):
        tmp1.append(words[i:i+num1])
        tmp2.append(words[i:i+num2])
        tmp3.append(words[i:i+num3])
        
    return tmp1,tmp2,tmp3

def diff_ngram_mul(text1, text2, num1,num2,num3):
    
    a1,a2,a3 = ngram_word_mul(text1, num1,num2,num3)
    b1,b2,b3 = ngram_word_mul(text2, num1,num2,num3) 
    
    cnt1 = 0 
    cnt2 = 0 
    cnt3 = 0 
    for i in a1:
        for j in b1:
            if i == j:
                cnt1 += 1
    for i in a2:
        for j in b2:
            if i == j:
                cnt2 += 1
    for i in a3:
        for j in b3:
            if i == j:
                cnt3 += 1
    return cnt1/len(a1),cnt2/len(a2),cnt3/len(a3)

In [10]:
def diff_ngram(text1, text2, num):
    
    a = ngram_word(text1, num)
    b = ngram_word(text2, num) 
    
    cnt = 0 
    for i in a:
        for j in b:
            if i == j:
                cnt += 1
                #common.append(i)
    if len(a) == 0:
        return 100000
    else:
        return cnt/len(a)


## n=2

In [2]:
feature_2gram = np.loadtxt('feature/feature_2gram.txt')

In [26]:
i = 11811
(diff_ngram(data.pair1[i], data.pair2[i], 2),feature_2gram[i])

(0.4873069876995551, 0.4873069876995551)

In [39]:
i = 11981
j = 12000
feature_2gram[i:j]

array([0.56009305, 0.4723779 , 0.64010485, 0.51603575, 0.45021299,
       0.54766786, 0.61609431, 0.51845208, 0.72571873, 0.57393293,
       0.70162107, 1.03834434, 1.01359517, 0.88978157, 0.94869215,
       0.66524681, 0.91368159, 0.16462958, 0.37136983])

In [30]:
target = pd.read_csv('data/info_truth.csv',encoding = 'utf-8')

In [32]:
feature_2gram[25814] = np.median(feature_2gram)

In [33]:
np.savetxt('feature/feature_2gram.txt',feature_2gram)

In [17]:
#25814
for i in range(len(feature_2gram)):
    if feature_2gram[i] == 100000:
        print(i)

25814


In [28]:
len(feature_2gram)

52601

In [287]:
i = 48621
diff_ngram(data.pair1[i], data.pair2[i], 2)

0.37493553378029915

In [ ]:
feature_2gram[25814] = np.max(feature_2gram) # 25814 change to 105.29237288135593

### n=3

In [189]:
%%time
diff_ngram(data.pair1[0], data.pair2[0], 2)

CPU times: user 587 ms, sys: 3.26 ms, total: 590 ms
Wall time: 590 ms


0.23336719146775012

In [217]:
%%time
diff_ngram(data.pair1[1], data.pair2[1], 3)

CPU times: user 610 ms, sys: 4.6 ms, total: 614 ms
Wall time: 619 ms


0.01828803657607315

In [216]:
%%time
diff_ngram_mul(data.pair1[0], data.pair2[0], 2,3,4)

CPU times: user 1.74 s, sys: 4.31 ms, total: 1.74 s
Wall time: 1.74 s


(0.23348577235772358, 0.008130081300813009, 0.0012703252032520325)

In [106]:
feature_3gram = np.zeros(len(data))
num = 3
for i in range(len(data)):
    feature_3gram[i] = diff_ngram(data.pair1[i], data.pair2[i], num)

In [225]:
#25814
for i in range(len(feature_3gram)):
    if feature_3gram[i] == 0:
        print(i)

11276
11278
11280
21579
21580
21581
21582
25814
28384
29345
30491
30492
30493
30494
30495
30496
30497
30498
30499
30500
30501
30502
30503
30504
30505
30506
30507
30508
30509
30510
30511
30512
30513
30514
30515
30516
30517
30518
30519
30520
30521
30522
30523
30524
30525
30526
30527
30528
30529
30530
30531
30532
30533
30534
30535
30536
30537
30538
30539
30540
30541
30542
30543
30544
30545
30546
30547
30548
30549
30550
30551
30552
30553
30554
30555
30556
30557
30558
30559
30560
30561
30562
30563
30564
30565
30566
30567
30568
30569
30570
30571
30572
30573
30574
30575
30576
30577
30578
30579
30580
30581
30582
30583
30584
30585
30586
30587
30588
30589
30590
30591
30592
30593
30594
30595
30596
30597
30598
30599
30600
30601
30602
30603
30604
30605
30606
30607
30608
30609
30610
30611
30612
30613
30614
30615
30616
30617
30618
30619
30620
30621
30622
30623
30624
30625
30626
30627
30628
30629
30630
30631
30632
30633
30634
30635
30636
30637
30638
30639
30640
30641
30642
30643
30644
30645
30646
3064

33750
33751
33752
33753
33754
33755
33756
33757
33758
33759
33760
33761
33762
33763
33764
33765
33766
33767
33768
33769
33770
33771
33772
33773
33774
33775
33776
33777
33778
33779
33780
33781
33782
33783
33784
33785
33786
33787
33788
33789
33790
33791
33792
33793
33794
33795
33796
33797
33798
33799
33800
33801
33802
33803
33804
33805
33806
33807
33808
33809
33810
33811
33812
33813
33814
33815
33816
33817
33818
33819
33820
33821
33822
33823
33824
33825
33826
33827
33828
33829
33830
33831
33832
33833
33834
33835
33836
33837
33838
33839
33840
33841
33842
33843
33844
33845
33846
33847
33848
33849
33850
33851
33852
33853
33854
33855
33856
33857
33858
33859
33860
33861
33862
33863
33864
33865
33866
33867
33868
33869
33870
33871
33872
33873
33874
33875
33876
33877
33878
33879
33880
33881
33882
33883
33884
33885
33886
33887
33888
33889
33890
33891
33892
33893
33894
33895
33896
33897
33898
33899
33900
33901
33902
33903
33904
33905
33906
33907
33908
33909
33910
33911
33912
33913
33914
33915
3391

37250
37251
37252
37253
37254
37255
37256
37257
37258
37259
37260
37261
37262
37263
37264
37265
37266
37267
37268
37269
37270
37271
37272
37273
37274
37275
37276
37277
37278
37279
37280
37281
37282
37283
37284
37285
37286
37287
37288
37289
37290
37291
37292
37293
37294
37295
37296
37297
37298
37299
37300
37301
37302
37303
37304
37305
37306
37307
37308
37309
37310
37311
37312
37313
37314
37315
37316
37317
37318
37319
37320
37321
37322
37323
37324
37325
37326
37327
37328
37329
37330
37331
37332
37333
37334
37335
37336
37337
37338
37339
37340
37341
37342
37343
37344
37345
37346
37347
37348
37349
37350
37351
37352
37353
37354
37355
37356
37357
37358
37359
37360
37361
37362
37363
37364
37365
37366
37367
37368
37369
37370
37371
37372
37373
37374
37375
37376
37377
37378
37379
37380
37381
37382
37383
37384
37385
37386
37387
37388
37389
37390
37391
37392
37393
37394
37395
37396
37397
37398
37399
37400
37401
37402
37403
37404
37405
37406
37407
37408
37409
37410
37411
37412
37413
37414
37415
3741

40568
40569
40570
40571
40572
40573
40574
40575
40576
40577
40578
40579
40580
40581
40582
40583
40584
40585
40586
40587
40588
40589
40590
40591
40592
40593
40594
40595
40596
40597
40598
40599
40600
40601
40602
40603
40604
40605
40606
40607
40608
40609
40610
40611
40612
40613
40614
40615
40616
40617
40618
40619
40620
40621
40622
40623
40624
40625
40626
40627
40628
40629
40630
40631
40632
40633
40634
40635
40636
40637
40638
40639
40640
40641
40642
40643
40644
40645
40646
40647
40648
40649
40650
40651
40652
40653
40654
40655
40656
40657
40658
40659
40660
40661
40662
40663
40664
40665
40666
40667
40668
40669
40670
40671
40672
40673
40674
40675
40676
40677
40678
40679
40680
40681
40682
40683
40684
40685
40686
40687
40688
40689
40690
40691
40692
40693
40694
40695
40696
40697
40698
40699
40700
40701
40702
40703
40704
40705
40706
40707
40708
40709
40710
40711
40712
40713
40714
40715
40716
40717
40718
40719
40720
40721
40722
40723
40724
40725
40726
40727
40728
40729
40730
40731
40732
40733
4073

43425
43426
43427
43428
43429
43430
43431
43432
43433
43434
43435
43436
43437
43438
43439
43440
43441
43442
43443
43444
43445
43446
43447
43448
43449
43450
43451
43452
43453
43454
43455
43456
43457
43458
43459
43460
43461
43462
43463
43464
43465
43466
43467
43468
43469
43470
43471
43472
43473
43474
43475
43476
43477
43478
43479
43480
43481
43482
43483
43484
43485
43486
43487
43488
43489
43490
43491
43492
43493
43494
43495
43496
43497
43498
43499
43500
43501
43502
43503
43504
43505
43506
43507
43508
43509
43510
43511
43512
43513
43514
43515
43516
43517
43518
43519
43520
43521
43522
43523
43524
43525
43526
43527
43528
43529
43530
43531
43532
43533
43534
43535
43536
43537
43538
43539
43540
43541
43542
43543
43544
43545
43546
43547
43548
43549
43550
43551
43552
43553
43554
43555
43556
43557
43558
43559
43560
43561
43562
43563
43564
43565
43566
43567
43568
43569
43570
43571
43572
43573
43574
43575
43576
43577
43578
43579
43580
43581
43582
43583
43584
43585
43586
43587
43588
43589
43590
4359

46713
46714
46715
46716
46717
46718
46719
46720
46721
46722
46723
46724
46725
46726
46727
46728
46729
46730
46731
46732
46733
46734
46735
46736
46737
46738
46739
46740
46741
46742
46743
46744
46745
46746
46747
46748
46749
46750
46751
46752
46753
46754
46755
46756
46757
46758
46759
46760
46761
46762
46763
46764
46765
46766
46767
46768
46769
46770
46771
46772
46773
46774
46775
46776
46777
46778
46779
46780
46781
46782
46783
46784
46785
46786
46787
46788
46789
46790
46791
46792
46793
46794
46795
46796
46797
46798
46799
46800
46801
46802
46803
46804
46805
46806
46807
46808
46809
46810
46811
46812
46813
46814
46815
46816
46817
46818
46819
46820
46821
46822
46823
46824
46825
46826
46827
46828
46829
46830
46831
46832
46833
46834
46835
46836
46837
46838
46839
46840
46841
46842
46843
46844
46845
46846
46847
46848
46849
46850
46851
46852
46853
46854
46855
46856
46857
46858
46859
46860
46861
46862
46863
46864
46865
46866
46867
46868
46869
46870
46871
46872
46873
46874
46875
46876
46877
46878
4687

49748
49749
49750
49751
49752
49753
49754
49755
49756
49757
49758
49759
49760
49761
49762
49763
49764
49765
49766
49767
49768
49769
49770
49771
49772
49773
49774
49775
49776
49777
49778
49779
49780
49781
49782
49783
49784
49785
49786
49787
49788
49789
49790
49791
49792
49793
49794
49795
49796
49797
49798
49799
49800
49801
49802
49803
49804
49805
49806
49807
49808
49809
49810
49811
49812
49813
49814
49815
49816
49817
49818
49819
49820
49821
49822
49823
49824
49825
49826
49827
49828
49829
49830
49831
49832
49833
49834
49835
49836
49837
49838
49839
49840
49841
49842
49843
49844
49845
49846
49847
49848
49849
49850
49851
49852
49853
49854
49855
49856
49857
49858
49859
49860
49861
49862
49863
49864
49865
49866
49867
49868
49869
49870
49871
49872
49873
49874
49875
49876
49877
49878
49879
49880
49881
49882
49883
49884
49885
49886
49887
49888
49889
49890
49891
49892
49893
49894
49895
49896
49897
49898
49899
49900
49901
49902
49903
49904
49905
49906
49907
49908
49909
49910
49911
49912
49913
4991

In [228]:
%%time
num = 3
for i in range(30491,len(data)):
    feature_3gram[i] = diff_ngram(data.pair1[i], data.pair2[i], num)
    

CPU times: user 3h 31min 18s, sys: 5.06 s, total: 3h 31min 23s
Wall time: 3h 31min 38s


In [242]:
np.max(feature_3gram)

105.29237288135593

In [243]:
feature_3gram[25814] = np.max(feature_3gram) # 25814 change to 105.29237288135593

In [245]:
#for #25814 # 30491
feature_3gram_error0 = []
for i in range(len(feature_3gram)):
    if feature_3gram[i] == 0:
        feature_3gram_error0.append(i)
        print(i)

11276
11278
11280
21579
21580
21581
21582
28384
29345
30624
31522
33002
33894
34608
37798
39730
40511
40625
40938
42372
42454
43277
44674
46164
46256
47163
49303
50039
50779
50830
52108


In [247]:
np.savetxt('feature/feature_3gram.txt',feature_3gram)

In [248]:
np.savetxt('feature/3gram_error0.txt',feature_3gram_error0)

## n=4

In [268]:
%%time
feature_4gram = np.zeros(len(data))
num = 4
for i in range(len(data)):
    feature_4gram[i] = diff_ngram(data.pair1[i], data.pair2[i], num)

CPU times: user 8h 26min 36s, sys: 18.7 s, total: 8h 26min 55s
Wall time: 8h 27min 34s


In [249]:
%%time
diff_ngram(data.pair1[1], data.pair2[1], 4)

CPU times: user 600 ms, sys: 3.85 ms, total: 603 ms
Wall time: 613 ms


0.0020325203252032522

In [271]:
np.savetxt('feature/feature_4gram.txt',feature_4gram)

## TF-IDF cosine similarity

In [100]:
def tfidf_cos(num):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(np.array([data.pair1[num],data.pair2[num]])).toarray()
    return cosine_similarity(tfidf)[0][1]

In [104]:
%%time
tfidf_cos(2)

CPU times: user 11.8 ms, sys: 1.67 ms, total: 13.4 ms
Wall time: 11.9 ms


0.7306151647998674

In [110]:
%%time
feature_tfidf_cos = np.zeros(len(data))
for i in range(len(data)):
    feature_tfidf_cos[i] = tfidf_cos(i)

CPU times: user 6min 53s, sys: 33.9 s, total: 7min 27s
Wall time: 7min 36s


In [111]:
np.savetxt('feature/feature_tfidf_cos.txt',feature_tfidf_cos)

## textual lexical richness measures

In [1]:
!pip install lexicalrichness

In [66]:
from lexicalrichness import LexicalRichness

In [81]:
%%time
#feature_lexical_richness = np.zeros(len(data))
richness12 = np.zeros((len(data),2))
for i in range(len(data)):
    richness1 = LexicalRichness(data.pair1[i]).mtld(threshold=0.72)
    richness2 = LexicalRichness(data.pair2[i]).mtld(threshold=0.72)
    richness12[i][0] = richness1
    richness12[i][1] = richness2

CPU times: user 5min 26s, sys: 1.13 s, total: 5min 27s
Wall time: 5min 32s


In [82]:
for i in range(len(data)):
    if richness12[i][0] == 0 or richness12[i][1] == 0:
        print(richness12[i])

In [83]:
np.savetxt('feature/richness12.txt',richness12)

In [87]:
%%time
feature_lexical_richness = np.zeros(len(data))
for i in range(len(data)):
    feature_lexical_richness[i] = np.max(richness12[i])/np.min(richness12[i])

CPU times: user 376 ms, sys: 27.6 ms, total: 404 ms
Wall time: 380 ms


In [88]:
np.savetxt('feature/feature_lexical_richness.txt',feature_lexical_richness)

In [89]:
%%time
feature_lexical_richness = np.zeros(len(data))
for i in range(len(data)):
    feature_lexical_richness[i] = np.min(richness12[i])/np.max(richness12[i])

CPU times: user 389 ms, sys: 17.5 ms, total: 407 ms
Wall time: 394 ms


In [90]:
np.savetxt('feature/feature_lexical_richness_min.txt',feature_lexical_richness)

In [79]:
%%time
i = 2
#Return Measure of Textual Lexical Diversity (MTLD).
#feature_lexical_richness = np.zeros(len(data))
richness12 = np.zeros((len(data),2))
richness1 = LexicalRichness(data.pair1[i]).mtld(threshold=0.72)
richness2 = LexicalRichness(data.pair2[i]).mtld(threshold=0.72)
richness12[i][0] = richness1
richness12[i][1] = richness2

CPU times: user 8.82 ms, sys: 133 µs, total: 8.96 ms
Wall time: 8.89 ms


## type-token ratio

In [ ]:
%%time 
# Return moving average type-token ratio (MATTR).
#feature_lexical_richness = np.zeros(len(data))
richness12 = np.zeros((len(data),2))
for i in range(len(data)):
    richness1 = LexicalRichness(data.pair1[i]).mattr(window_size=25)
    richness2 = LexicalRichness(data.pair2[i]).mattr(window_size=25)
    richness12[i][0] = richness1
    richness12[i][1] = richness2

In [91]:
%%time
i = 0
richness1 = LexicalRichness(data.pair1[i]).mattr(window_size=25)
richness2 = LexicalRichness(data.pair2[i]).mattr(window_size=25)
richness12[i][0] = richness1
richness12[i][1] = richness2

CPU times: user 17 ms, sys: 37.7 ms, total: 54.7 ms
Wall time: 132 ms


## richness yule k

In [90]:
from nltk.stem.porter import PorterStemmer
from itertools import groupby

In [86]:
# code from https://swizec.com/blog/measuring-vocabulary-richness-with-python/

def words(entry):
    return filter(lambda w: len(w) > 0,
                  [w.strip("0123456789!:,.?(){}[]") for w in entry.split()])

def yule(entry):
    # yule's I measure (the inverse of yule's K measure)
    # higher number is higher diversity - richer vocabulary
    d = {}
    stemmer = PorterStemmer()
    for w in words(entry):
        w = stemmer.stem(w).lower()
        try:
            d[w] += 1
        except KeyError:
            d[w] = 1

    M1 = float(len(d)) #unique word number
    M2 = sum([len(list(g))*(freq**2) for freq,g in groupby(sorted(d.values()))]) #将词频相同的词合并，合并的单词个数*频率的2倍
    
    try:
        return (M1*M1)/(M2-M1)
    except ZeroDivisionError:
        return 0

In [91]:
d = {}
stemmer = PorterStemmer()

    
for w in words(data.pair1[0]):
    w = stemmer.stem(w).lower()
    try:
        d[w] += 1
    except KeyError:
        d[w] = 1

In [100]:
for freq,g in groupby([1,2,2,2,3,4,4,5,9]):
    print(freq)
    print(len(list(g)))

1
1
2
3
3
1
4
2
5
1
9
1


In [101]:
sum([len(list(g))*(freq**2) for freq,g in groupby(sorted([1,2,3,3,4,4,4]))])

[1, 4, 18, 48]

In [102]:
sum([len(list(g))*(freq**2) for freq,g in groupby(sorted([1,2,3,3,4,4,4]))])

71

In [103]:
sum([len(list(g))*(freq**2) for freq,g in groupby(sorted([1,2,3,4,5,6,7]))])

140

In [24]:
%%time
i = 0
yule12 = np.zeros((len(data),2))
yule1 = yule(data.pair1[0])
yule2 = yule(data.pair2[0])
yule12[i][0] = yule1
yule12[i][1] = yule2

CPU times: user 153 ms, sys: 2.39 ms, total: 155 ms
Wall time: 155 ms


In [26]:
%%time
yule12 = np.zeros((len(data),2))
for i in range(len(data)):
    yule1 = yule(data.pair1[i])
    yule2 = yule(data.pair2[i])
    yule12[i][0] = yule1
    yule12[i][1] = yule2

CPU times: user 1h 55min 3s, sys: 2.94 s, total: 1h 55min 6s
Wall time: 1h 55min 15s


In [44]:
# change
yule12[25814][0] = yule12[25814][1]

In [28]:
for i in range(len(yule12)):
    if yule12[i][0] == 0 or yule12[i][1] == 0:
        print(i)

25814


In [50]:
np.max(yule12[25814])/np.min(yule12[25814])

1.0

In [53]:
np.savetxt('feature/yule12.txt',yule12)

In [51]:
%%time
feature_yule = np.zeros(len(data))
for i in range(len(data)):
    feature_yule[i] = np.max(yule12[i])/np.min(yule12[i])

CPU times: user 381 ms, sys: 12.6 ms, total: 393 ms
Wall time: 384 ms


In [54]:
np.savetxt('feature/feature_yule.txt',feature_yule)